In [15]:
import codecs, json, random
from pathlib import Path
import asyncio
from crawl4ai import *
from tqdm import tqdm
import warnings
import time
import datetime


In [16]:
BASE = Path().cwd()
BASE


PosixPath('/Users/white/shad/Scrapper')

In [17]:
RESOURCES_DIR = BASE.joinpath("resources")
SCRAPPED_DATA_DIR = BASE.joinpath("scrapped_data")

print(f"isdir({RESOURCES_DIR}) = {RESOURCES_DIR.is_dir()}")
print(f"isdir({SCRAPPED_DATA_DIR}) = {SCRAPPED_DATA_DIR.is_dir()}")


isdir(/Users/white/shad/Scrapper/resources) = True
isdir(/Users/white/shad/Scrapper/scrapped_data) = True


In [18]:
url_fname = RESOURCES_DIR.joinpath("web_urls.json")
print(f"isfile({url_fname}) = {url_fname.is_file()}")


isfile(/Users/white/shad/Scrapper/resources/web_urls.json) = True


In [19]:
with url_fname.open(mode="r", encoding="utf-8", errors="ignore") as fp:
    url_data = json.load(fp)


In [20]:
url_dict = dict()
for doc_name in url_data:
    doc_url = url_data[doc_name]
    if doc_url in url_dict:
        url_dict[doc_url].add(doc_name)
    else:
        url_dict[doc_url] = {doc_name}
for doc_url in url_dict:
    name_variants = sorted(list(url_dict[doc_url]), key=lambda it: -len(it))
    url_dict[doc_url] = " ".join(name_variants[0].strip().split()).strip()


In [21]:
del url_data


In [22]:
print(f"There are {len(url_dict)} documents.")


There are 1283 documents.


In [23]:
url_list = sorted(list(url_dict.keys()))


In [24]:
for doc_url in random.sample(population=url_list, k=3):
    print(doc_url)
    print(url_dict[doc_url])
    print("")


https://fen.nsu.ru/fen.phtml?topic=specc
Структура кафедр факультета Естественных Наук на осенний и весенний семестры

https://www.nsu.ru/n/economics-department/prepodavateli/3540521/
Профессия экономиста в нефтегазовой промышленности: Образование, исследования и достижения

https://education.nsu.ru/journalism_master_producing/
Магистратура Новосибирского государственного университета: Искусство и технологии видеопроизводства



In [25]:
browser_config = BrowserConfig(verbose=False)
run_config = CrawlerRunConfig(
    markdown_generator=DefaultMarkdownGenerator(
        content_filter=PruningContentFilter(threshold=0.6),
        options={"ignore_links": True},
    ),
    word_count_threshold=10,  # Minimum words per content block
    excluded_tags=["form", "header"],
    exclude_external_links=True,  # Remove external links
    remove_overlay_elements=True,  # Remove popups/modals
    process_iframes=True,
    verbose=False,
)


In [26]:
async def main():
    # 1. Формируем имя файла с текущей датой
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")
    filename = f"web_scrapped_{current_date}.jsonl"
    file_path = SCRAPPED_DATA_DIR.joinpath(filename)

    print(f"Saving data to {file_path}")

    success_count = 0
    fail_count = 0

    # 2. Сбор данных
    with open(file_path, mode="w", encoding="utf-8") as fp:
        async with AsyncWebCrawler(config=browser_config) as crawler:
            for doc_url in tqdm(url_list):
                try:
                    result = await crawler.arun(url=doc_url, config=run_config)

                    if result.success:
                        jsonified_result = {
                            "url": doc_url,
                            "name": url_dict[doc_url],
                            "content": result.markdown.fit_markdown,
                            "date": None,  # Для веб-страниц часто нет явной даты публикации
                            "collection_date": int(time.time()),
                        }
                        fp.write(
                            json.dumps(jsonified_result, ensure_ascii=False) + "\n"
                        )
                        fp.flush()  # Сохраняем сразу
                        success_count += 1
                    else:
                        fail_count += 1
                        # Генерируем предупреждение, но не останавливаем скрипт
                        warnings.warn(
                            f"FAIL {doc_url}: Status={result.status_code}, Error={result.error_message}"
                        )
                except Exception as e:
                    fail_count += 1
                    print(f"EXCEPTION {doc_url}: {e}")

    # 3. Итоговый отчет
    print("-" * 40)
    print(f"🎉 Готово!")
    print(f"Всего URLs: {len(url_list)}")
    print(f"✅ Успешно: {success_count}")
    if fail_count > 0:
        print(f"⚠️ Ошибок: {fail_count} (см. предупреждения выше)")
    else:
        print(f"Ошибок: 0")
    print(f"Файл: {file_path}")


In [27]:
await main()


Saving data to /Users/white/shad/Scrapper/scrapped_data/web_scrapped_2026-01-20.jsonl


  8%|▊         | 106/1283 [03:19<36:51,  1.88s/it] /var/folders/dt/xh7q8b8n6mb7n93ylggbssp40000gn/T/ipykernel_13394/3165899271.py:35: UserWarning: FAIL http://www.ict.nsc.ru/: Status=None, Error=Unexpected error in _crawl_web at line 718 in _crawl_web (.venv/lib/python3.13/site-packages/crawl4ai/async_crawler_strategy.py):
Error: Failed on navigating ACS-GOTO:
Page.goto: Timeout 60000ms exceeded.
Call log:
  - navigating to "http://www.ict.nsc.ru/", waiting until "domcontentloaded"


Code context:
 713                                   tag="GOTO",
 714                                   params={"url": url},
 715                               )
 716                               response = None
 717                           else:
 718 →                             raise RuntimeError(f"Failed on navigating ACS-GOTO:\n{str(e)}")
 719   
 720                       # ──────────────────────────────────────────────────────────────
 721                       # Walk the redirect chain.  Playwri

----------------------------------------
🎉 Готово!
Всего URLs: 1283
✅ Успешно: 1265
⚠️ Ошибок: 18 (см. предупреждения выше)
Файл: /Users/white/shad/Scrapper/scrapped_data/web_scrapped_2026-01-20.jsonl
